In [4]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from mutation import MyMutation
from crossover import TraceCrossover
from encoder import Encoder
from sampling import MySampling
from callback import UpdatePopulationCallback, UpdatePopCallback
import numpy as np
from tools import Tools
from survival import MySurvival
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
import pandas as pd
import logging
from terminator import MyTermination
from problem import Problem_single_ElementWise, MyProblem_Problem, MyProblem_Problem2, Problem_multi_ElementWise, Problem_single_ElementWise_noConstraints
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination
import testSetup
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.crossover.sbx import SBX
from mutation import IntegerPolynomialMutation




logging.getLogger('matplotlib').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", ".*feasible.*")


# declare = DeclareModel().parse_from_file("../declare_models/model1.decl")
# model_constraints = declare.get_decl_model_constraints()
# activities_name = declare.get_model_activities()
#
# encoder = Encoder(activities_name)
#
# variable_boundaries = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
# timestamps = Tools.generate_random_timestamps(len(activities_name))
# case_concept_name = ['1'] * len(activities_name)
#
# data = {
#     'case:concept:name': case_concept_name,
#     'concept:name': activities_name,
#     'timestamp': timestamps,
# }
#
# dataframe = pd.DataFrame(data)
#
#
# dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'])




# create random pop
n_traces = 100
trace_length = 20
variable_boundaries = [1] * trace_length

trace = [
    "Return ER", "LacticAcid", "Return ER", "IV Liquid", "ER Triage",
    "ER Sepsis Triage", "IV Antibiotics", "LacticAcid", "Admission NC",
    "Release B", "IV Liquid", "LacticAcid", "ER Registration",
    "ER Sepsis Triage", "Admission NC", "Admission IC", "LacticAcid",
    "ER Sepsis Triage", "LacticAcid", "Release A"
]

declare = DeclareModel().parse_from_file("../declare_models/model4.decl")

activities_name = declare.get_model_activities()

timestamps = Tools.generate_random_timestamps(trace_length)

data = {
    'case:concept:name': ['1'] * trace_length,
    'concept:name': ['1'] * trace_length,
    'timestamp': pd.to_datetime(timestamps),
}

dataframe = pd.DataFrame(data)

event_log = D4PyEventLog()
event_log.log = dataframe
event_log.timestamp_key = "timestamp"
event_log.activity_key = "concept:name"

event_log.to_eventlog()

model_constraints = declare.get_decl_model_constraints()

encoder = Encoder(activities_name)

print(activities_name)


['Release B', 'ER Triage', 'ER Registration', 'ER Sepsis Triage', 'Leucocytes', 'CRP', 'LacticAcid', 'IV Antibiotics', 'Admission NC', 'IV Liquid', 'Release A', 'Return ER', 'Admission IC']


In [5]:




from pymoo.termination.default import DefaultMultiObjectiveTermination

initial_population = [[random.choice(activities_name) for _ in range(trace_length)] for _ in range(n_traces)]
print(initial_population)
initial_encoded_pop = [encoder.encode(trace) for trace in initial_population]

features_range = Tools.calculate_feature_range(initial_encoded_pop, variable_boundaries)
# lower_bounds = [x[0] for x in features_range]
# upper_bounds = [x[1] for x in features_range]
lower_bounds = 0
upper_bounds = len(activities_name) - 1


# mutation = MyMutation(feature_range=features_range)
# mutation = PolynomialMutation(prob=0.1, eta=20)
mutation = IntegerPolynomialMutation(prob=0.4,eta=20) # low eta (5–10) → More exploration - high eta (15–50) → More exploitation
# crossover = TraceCrossover(variable_boundaries=variable_boundaries)
crossover = SBX(prob=0.9, eta=15)
sampling = MySampling(initial_population=initial_encoded_pop)
pop_size = 2000
survival = MySurvival(n_children_survive=50)

termination_multi = DefaultMultiObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=0.0025,
    period=30,
    n_max_gen=1000,
    n_max_evals=100000
)

termination_single = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=1000,
    n_max_evals=100000
)


termination = MyTermination(n_required=pop_size)
# termination = get_termination("n_gen", 10)
# termination1 = termination
# termination2 = termination
print('\n----------------------------------------------\n')
print(len(trace))
print('\n----------------------------------------------\n')
print(trace_length)

[['IV Liquid', 'ER Triage', 'CRP', 'Release B', 'Release B', 'Release B', 'Release A', 'Admission NC', 'Release B', 'LacticAcid', 'Release A', 'ER Sepsis Triage', 'LacticAcid', 'Return ER', 'Release B', 'Admission NC', 'ER Sepsis Triage', 'Admission IC', 'IV Antibiotics', 'IV Antibiotics'], ['Admission NC', 'ER Sepsis Triage', 'CRP', 'ER Sepsis Triage', 'Release A', 'ER Sepsis Triage', 'Admission IC', 'IV Antibiotics', 'Leucocytes', 'Release B', 'LacticAcid', 'Admission NC', 'Release A', 'ER Triage', 'ER Registration', 'Release A', 'Return ER', 'Leucocytes', 'ER Triage', 'Return ER'], ['CRP', 'Return ER', 'Return ER', 'Admission NC', 'LacticAcid', 'Admission NC', 'Release A', 'ER Sepsis Triage', 'Leucocytes', 'Leucocytes', 'IV Liquid', 'IV Antibiotics', 'Admission NC', 'LacticAcid', 'IV Liquid', 'Release B', 'IV Antibiotics', 'ER Sepsis Triage', 'Return ER', 'Admission IC'], ['LacticAcid', 'LacticAcid', 'Release A', 'ER Registration', 'CRP', 'Admission NC', 'Return ER', 'Admission IC',

In [ ]:
from testSetup import Setup


# (encoder, declare, event_log, dataframe, activities_name) = Setup.initialize_shared_components("../declare_models/model1.decl", trace_length)


multi_problem = Problem_multi_ElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

single_problem = Problem_single_ElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)


multi_algorithm = NSGA2(
    problem=multi_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(problem=multi_problem),
    eliminate_duplicates=False,
)

single_algorithm = GA(
    problem=single_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(problem=single_problem),
    eliminate_duplicates=False,
)


result = minimize(single_problem, single_algorithm, termination=termination, seed=1, verbose=True)
data = result.algorithm.callback.get_data()
diversity_scores = data.get("diversity_history", None)
constraint_scores = data.get("constraint_history", None)
n_violations_scores = data.get("n_violations_history", None)
n_generations = data.get("generations", None)
print(n_generations)
print(constraint_scores)


Setup.plot_progress(diversity_scores=diversity_scores, constraint_scores=constraint_scores, n_violations_scores=n_violations_scores, n_generations=n_generations)

G = np.array([individual.G for individual in result.pop])
traces = [individual.X.tolist() for individual in result.pop]

# Print the feasible traces
for i, (trace, g) in enumerate(zip(traces, G)):
    if g == 0:
        decoded_trace =  encoder.decode(trace)

        dataframe['concept:name'] = pd.DataFrame(decoded_trace)
        event_log.log = dataframe
        event_log.to_eventlog()

        basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=declare, consider_vacuity=False)
        conf_check_res: MPDeclareResultsBrowser = basic_checker.run()
        print(f"Feasible Trace {i + 1}: {trace}")

        print("Solution Decoded:", encoder.decode(trace))
        print(conf_check_res.get_metric(trace_id=0, metric="num_violations"))
        print("-------------------------------------------")


n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  1.500000E+01 |  2.045000E+01 |             - |             -
     2 |     2100 |  1.100000E+01 |  1.953950E+01 |             - |             -
     3 |     4100 |  1.100000E+01 |  1.731800E+01 |             - |             -
     4 |     6100 |  1.100000E+01 |  1.629550E+01 |             - |             -
     5 |     8100 |  1.100000E+01 |  1.552750E+01 |             - |             -
     6 |    10100 |  1.000000E+01 |  1.499050E+01 |             - |             -
     7 |    12100 |  1.000000E+01 |  1.450150E+01 |             - |             -
     8 |    14100 |  1.000000E+01 |  1.417000E+01 |             - |             -
     9 |    16100 |  1.000000E+01 |  1.382750E+01 |             - |             -
    10 |    18100 |  1.000000E+01 |  1.341300E+01 |             - |             -
    11 |    20100 |  1.000000E+01 |  1.318300E+01 |             - |             -
    12 |    2210

In [ ]:
problem = Problem_single_ElementWise_noConstraints(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

callback = UpdatePopCallback(problem=problem)

algorithm = GA(
    problem=problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=callback,
    eliminate_duplicates=False,
)


result = minimize(problem, algorithm, termination=termination, seed=1, verbose=True)

traces = [individual.X.tolist() for individual in result.pop]


print("Best Solution Encoded:", result.X)
# print("Best Solution Decoded:", [encoder.decode(solution) for solution in result.X])
print("Objective Values:", result.F)


n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       10 | -2.470000E+01 | -2.540000E+01
     2 |     2010 | -2.561115E+01 | -2.720000E+01
     3 |     4010 | -2.637070E+01 | -2.760000E+01
     4 |     6010 | -2.688490E+01 | -2.800000E+01
     5 |     8010 | -2.731465E+01 | -2.830000E+01
     6 |    10010 | -2.765330E+01 | -2.870000E+01
     7 |    12010 | -2.792565E+01 | -2.870000E+01
     8 |    14010 | -2.814975E+01 | -2.890000E+01
     9 |    16010 | -2.832805E+01 | -2.920000E+01
    10 |    18010 | -2.848685E+01 | -2.930000E+01
    11 |    20010 | -2.862655E+01 | -2.930000E+01
    12 |    22010 | -2.874495E+01 | -2.950000E+01
    13 |    24010 | -2.885395E+01 | -2.950000E+01
    14 |    26010 | -2.895105E+01 | -2.960000E+01
    15 |    28010 | -2.904135E+01 | -2.960000E+01
    16 |    30010 | -2.912185E+01 | -2.960000E+01
    17 |    32010 | -2.919800E+01 | -2.990000E+01
    18 |    34010 | -2.925685E+01 | -2.990000E+01
    19 |    36010 | -2.932155E+01 | -2.990000E+01


In [17]:



problem = Problem_single_ElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

callback = UpdatePopulationCallback(problem=problem, plot=0)

algorithm = GA(
    problem=problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=callback,
    termination=termination,
    eliminate_duplicates=False,
)


result = minimize(problem, algorithm, termination=termination2, seed=1, verbose=True)

G = np.array([individual.G for individual in result.pop])
traces = [individual.X.tolist() for individual in result.pop]

# Print the feasible traces
for i, (trace, g) in enumerate(zip(traces, G)):
    if g == 0:
        print(f"Feasible Trace {i + 1}: {trace}")

print("Best Solution Encoded:", result.X)
# print("Best Solution Decoded:", [encoder.decode(solution) for solution in result.X])
print("Objective Values:", result.F)
print("Constraint Values:", result.G)


TypeError: UpdatePopulationCallback.__init__() got an unexpected keyword argument 'plot'

only few millisecond faster than the 2 obj algorithm

In [8]:



problem = MyProblem_Problem(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

callback = UpdatePopulationCallback(problem=problem, plot=0)


algorithm = NSGA2(
    problem=problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=callback,
    termination=termination,
    eliminate_duplicates=False,
)

result = minimize(problem, algorithm, termination=termination1, seed=1, verbose=True)


print("Best Solution Encoded:", result.X)
print("Best Solution Decoded:", [encoder.decode(solution) for solution in result.X])
print("Objective Values:", result.F)
print("Constraint Values:", result.G)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       10 |      1 |  0.3333333333 |  0.7666666667 |             - |             -
     2 |     2010 |      1 |  0.000000E+00 |  0.6755833333 |             - |             -
     3 |     4010 |      1 |  0.000000E+00 |  0.5208333333 |  0.4000000000 |         ideal
     4 |     6010 |      1 |  0.000000E+00 |  0.4137500000 |  0.000000E+00 |             f
     5 |     8010 |      2 |  0.000000E+00 |  0.3443333333 |  0.000000E+00 |             f
     6 |    10010 |      1 |  0.000000E+00 |  0.2790000000 |  0.2000000000 |         ideal
     7 |    12010 |      5 |  0.000000E+00 |  0.2414166667 |  0.1000000000 |         ideal
     8 |    14010 |      1 |  0.000000E+00 |  0.1887500000 |  0.4000000000 |         ideal
     9 |    16010 |      1 |  0.000000E+00 |  0.1365000000 |  0.1000000000 |         ideal
    10 |    18010 |      1 |  0.000000E+00 |  0.1208333333 |  0.000000E+00 |             f

Seems a little slower than with ElementalWise problem object

In [9]:



problem = MyProblem_Problem2(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

callback = UpdatePopulationCallback(problem=problem, plot=0)

algorithm = GA(
    problem=problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=callback,
    eliminate_duplicates=False,
)

result = minimize(problem, algorithm, termination=termination2, seed=1, verbose=True)

print("Best Solution Encoded:", result.X)
# print("Best Solution Decoded:", [encoder.decode(solution) for solution in result.X])
print("Objective Values:", result.F)
print("Constraint Values:", result.G)


n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  0.3333333333 |  0.7666666667 |             - |             -
     2 |     2010 |  0.1666666667 |  0.6750833333 |             - |             -
     3 |     4010 |  0.000000E+00 |  0.5258333333 | -1.790000E+01 | -1.790000E+01
     4 |     6010 |  0.000000E+00 |  0.4196666667 | -1.773333E+01 | -1.790000E+01
     5 |     8010 |  0.000000E+00 |  0.3510000000 | -1.774211E+01 | -1.820000E+01
     6 |    10010 |  0.000000E+00 |  0.2821666667 | -1.785918E+01 | -1.850000E+01
     7 |    12010 |  0.000000E+00 |  0.2451666667 | -1.788387E+01 | -1.860000E+01
     8 |    14010 |  0.000000E+00 |  0.1990000000 | -1.793148E+01 | -1.900000E+01
     9 |    16010 |  0.000000E+00 |  0.1422500000 | -1.797440E+01 | -1.900000E+01
    10 |    18010 |  0.000000E+00 |  0.1280833333 | -1.801793E+01 | -1.900000E+01
    11 |    20010 |  0.000000E+00 |  0.1108333333 | -1.807164E+01 | -1.900000E+01
    12 |    2201